In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold


In [2]:
df = pd.read_csv('../dataset/Auto.csv')
df['horsepower'] =df['horsepower'].replace('?',0)
df['horsepower'] =df['horsepower'].astype(int)
y = df['displacement']

del df['name']

In [3]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


In [4]:
df.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower        int64
weight            int64
acceleration    float64
year              int64
origin            int64
dtype: object

In [5]:
#split it into train and test 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(df, y,test_size = 0.33 , random_state = 1)

X_train = pd.DataFrame(X_train, columns = df.columns)
X_test = pd.DataFrame(X_test, columns = df.columns)


In [34]:
#Create dataset of lightgbm

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

#get number of feature and number of training rows
num_train, num_feature = X_train.shape

#Get parameters 
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


#------------------------------------------------------------
folds = KFold(n_splits=2, shuffle=True, random_state=123)

oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])

feature_name = [col for col in X_train.columns]
print(feature_name)
print('Starting training...')
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(X_train)):
    print('fold {} '.format(n_fold))
    trn_x, trn_y = X_train[feature_name].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = X_train[feature_name].iloc[val_idx], y.iloc[val_idx]
    lgb_train = lgb.Dataset(trn_x, trn_y)
    lgb_eval = lgb.Dataset(val_x, val_y)
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                valid_sets=[lgb_train,lgb_eval],
                valid_names=['train', 'valid'],# eval training data
                feature_name=feature_name,
                categorical_feature=[21])
    oof_preds[val_idx] = gbm.predict(val_x, num_iteration=gbm.best_iteration)
    sub_preds += gbm.predict(X_test[feature_name], num_iteration=gbm.best_iteration) / folds.n_splits 


# print('Starting training...')
# # feature_name and categorical_feature
# gbm = lgb.train(params,
#                 lgb_train,
#                 num_boost_round=10,
#                 valid_sets=lgb_train,  # eval training data
#                 feature_name=feature_name,
#                 categorical_feature=[21])

['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']
Starting training...
fold 0 
[1]	train's rmse: 108.301	valid's rmse: 113.378
[2]	train's rmse: 107.827	valid's rmse: 113.277
[3]	train's rmse: 107.558	valid's rmse: 113.426
[4]	train's rmse: 107.349	valid's rmse: 113.543
[5]	train's rmse: 106.788	valid's rmse: 113.754
[6]	train's rmse: 106.435	valid's rmse: 114.235
[7]	train's rmse: 106.122	valid's rmse: 114.713
[8]	train's rmse: 105.845	valid's rmse: 115.183
[9]	train's rmse: 105.562	valid's rmse: 115.559
[10]	train's rmse: 105.352	valid's rmse: 116.06
fold 1 
[1]	train's rmse: 110.464	valid's rmse: 111.153
[2]	train's rmse: 109.642	valid's rmse: 111.231
[3]	train's rmse: 109.144	valid's rmse: 111.299
[4]	train's rmse: 108.424	valid's rmse: 111.434
[5]	train's rmse: 107.994	valid's rmse: 111.543
[6]	train's rmse: 107.415	valid's rmse: 111.781
[7]	train's rmse: 107.014	valid's rmse: 111.93
[8]	train's rmse: 106.643	valid's rmse: 112
[9]	train